In [ ]:
# Colab 환경에서
# from google.colab import drive
# drive.mount('/content/drive')

## 데이터 불러오기

In [8]:
from pathlib import Path

current_path = Path().cwd()
root = current_path.parent.parent
data = root / 'data'
src = root / 'src'
images = root / 'images'
print(root.absolute())

/home/yeeun/Sparta/Project1


In [15]:
import pandas as pd

datasets = {
    'games_details': None,
    'ranking': None,
    'games': None,
    'players': None,
    'teams': None
}

pandas_kwargs = {
    'games_details': {
        'dtype': {'NICKNAME': str}
    },
    'ranking': {
        
    },
    'games': {
        
    },
    'players': {
        
    },
    'teams': {
        
    }
}

for name in datasets:
    datasets[name] = pd.read_csv(data / f'{name}.csv', **pandas_kwargs[name])

games_details = datasets['games_details']
games = datasets['games']
players = datasets['players']
ranking = datasets['ranking']
teams = datasets['teams']

In [25]:
ranking.head()

,TEAM_ID,LEAGUE_ID,SEASON_ID,STANDINGSDATE,CONFERENCE,TEAM,G,W,L,W_PCT,HOME_RECORD,ROAD_RECORD,RETURNTOPLAY
0,1610612743,0,22022,2022-12-22,West,Denver,30,19,11,0.633,10-3,9-8,NaN
1,1610612763,0,22022,2022-12-22,West,Memphis,30,19,11,0.633,13-2,6-9,NaN
2,1610612740,0,22022,2022-12-22,West,New Orleans,31,19,12,0.613,13-4,6-8,NaN
3,1610612756,0,22022,2022-12-22,West,Phoenix,32,19,13,0.594,14-4,5-9,NaN
4,1610612746,0,22022,2022-12-22,West,LA Clippers,33,19,14,0.576,11-7,8-7,NaN


## ranking 데이터 구조 분석

### ranking의 기본 정보

In [17]:
ranking.shape

(210342, 13)

In [18]:
ranking.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 210342 entries, 0 to 210341
Data columns (total 13 columns):
 #   Column         Non-Null Count   Dtype  
---  ------         --------------   -----  
 0   TEAM_ID        210342 non-null  int64  
 1   LEAGUE_ID      210342 non-null  int64  
 2   SEASON_ID      210342 non-null  int64  
 3   STANDINGSDATE  210342 non-null  object 
 4   CONFERENCE     210342 non-null  object 
 5   TEAM           210342 non-null  object 
 6   G              210342 non-null  int64  
 7   W              210342 non-null  int64  
 8   L              210342 non-null  int64  
 9   W_PCT          210342 non-null  float64
 10  HOME_RECORD    210342 non-null  object 
 11  ROAD_RECORD    210342 non-null  object 
 12  RETURNTOPLAY   3990 non-null    float64
dtypes: float64(2), int64(6), object(5)
memory usage: 20.9+ MB


### ranking의 고유값 체크

In [63]:
ranking.nunique()

TEAM_ID            30
LEAGUE_ID           1
SEASON_ID          41
STANDINGSDATE    7023
CONFERENCE          2
TEAM               34
G                  83
W                  74
L                  73
W_PCT             895
HOME_RECORD       800
ROAD_RECORD       758
RETURNTOPLAY        2
dtype: int64

In [ ]:
team_id_cond = ranking['TEAM_ID'] == 1610612743

ranking[team_id_cond]['TEAM'].value_counts()

TEAM
Denver    7024
Name: count, dtype: int64

In [42]:
grouped = ranking.groupby(['TEAM_ID']).agg(DISTINCT_TEAM=('TEAM', 'nunique'))

In [57]:
team_cond = (grouped[grouped['DISTINCT_TEAM'] > 1].index)
filtered = ranking[ranking['TEAM_ID'].isin(team_cond)].sort_values('STANDINGSDATE')
# ranking[team_cond]

In [60]:
print(filtered.groupby(['TEAM_ID']).agg(DISTINCT_TEAM=('TEAM', 'unique')))

                                       DISTINCT_TEAM
TEAM_ID                                             
1610612740  [New Orleans, New Orleans/Oklahoma City]
1610612746              [L.A. Clippers, LA Clippers]
1610612751                    [New Jersey, Brooklyn]
1610612760                  [Seattle, Oklahoma City]


In [62]:
seattle = ranking['TEAM'] == 'Seattle'
ranking[seattle]['TEAM_ID'].value_counts()

TEAM_ID
1610612760    1831
Name: count, dtype: int64